# 资源

1) http://educationaldatamining.org/files/conferences/EDM2018/papers/EDM2018_paper_139.pdf  采用红黑树的思想，结合BKT减少学习时间

2) https://www.researchgate.net/publication/220807585_Estimating_programming_knowledge_with_Bayesian_knowledge_tracing?_sg=LTDqpx9gm8Cz5OXfgjBfZQKuytPQJdh83M9h43tGZiFZKxnWotrND4fCdry9v4dPvpLJCuYmYcOKTAkEuZFQdlHjI9A49c6n1ucWIMUUoebu6Q

3) https://www.semanticscholar.org/paper/Finding-Similar-Exercises-in-Online-Education-Liu-Huang/41c828743f69c0ec53f59c42704f427ac879613b

4) https://www.semanticscholar.org/paper/Knowledge-tracing-with-an-intelligent-agent%2C-in-an-Trifa-Sba%C3%AF/fda25ab608702fcdb8ce1f259d55a7ab576e15b0

5) http://www.educationaldatamining.org/EDM2017/proc_files/papers/paper_72.pdf  通过Markov Chains预测用户下一个行为，改用LSTM更为方便，需要大量历史数据进行训练。

6) https://arxiv.org/pdf/1811.03388.pdf  Factorization Machines for Knowledge Tracing , 可用于评估学生能力  P(x) = 能力 - 试题难度. 开源软件 -->  KMT, pywFM

7) https://www.docin.com/p-764417569.html?docfrom=rrela 基于贝叶斯网络的NBA比分预测和球员能力评估模型, 包括了模型的算法一些总结

8) http://www.columbia.edu/~rsb2162/paper_143.pdf  BTK 训练好的参数的应用

9）https://www2.eecs.berkeley.edu/Pubs/TechRpts/2015/EECS-2015-98.pdf

10) doc: 397-Article Text-1853-1-10-20190624.pdf

11） https://blog.csdn.net/tanzhangwen/article/details/8262017  动态贝叶斯网络DBN

12) https://arxiv.org/pdf/1806.02180.pdf  Addressing Two Problems in Deep Knowledge Tracing viaPrediction-Consistent Regularization

13) doc:  https://www.cs.cmu.edu/~zhitingh/data/lats16structured.pdf  Structured Knowledge Tracing Models for Student Assessment on Coursera

14) https://www.freecodecamp.org/news/adaptive-learning-systems/ How to build an adaptive learning system

15) http://pact.cs.cmu.edu/pubs/New%20potentials%20for%20ITS-source.pdf  1New Potentials for Data-Driven Intelligent Tutoring System Development and Optimization 

16) https://crowston.syr.edu/sites/crowston.syr.edu/files/training%20v3%20to%20share_0.pdf

17) https://www.zhihu.com/question/60893517 如何理解odds即p/(1-p)在数据预处理中的应用？odds即赔率，是一个0到正无穷的实数值，相比于 [0, 1] 的概率值具有更大的范围。同时，赔率理解起来更直观，假设中国队世界杯夺冠的概率为 p = 0.01，大多数人对概率没有直观感觉。但如果换成赔率为 1/99，其所表达的意思为如果赌博中国队世界杯夺冠100次，会输掉99次。

作者：赵熙
链接：https://www.zhihu.com/question/60893517/answer/183083929
来源：知乎
著作权归作者所有。商业转载请联系作者获得授权，非商业转载请注明出处。

红黑树

bing : Intelligent Scheduler student study Schedules
bing : bayesian knowledge tracing how to evaluate learning materials   transform
bing : ITS how to evaluate learning materials transform

P(L0): 预计掌握情况（可通过预测试方式得到）
P(Ln): 通过后的掌握情况
P(T):  转换情况
P(G), P(S): 根据题型来选择

1、知识点测试结果, 数据格式说明：
    student 1: {q0, a0}, {q1, a1}, {q2, a2} ... {qn, an}  -->  q0  试题编号（后期可扩展到基本内容等方面）
    
   


# 数据准备

In [76]:
import numpy as np
# data_path = './data/train_data.npy'
# train_data = np.load(data_path, allow_pickle=True)
# # fast-correct, fast-incorrect, slow-correct, slow-incorrect
# train_feature = [[0,1,2,3]]
np.random.seed(1) 
gen_number = 1000
exam_number = 10
feature = [0,1]  # 0: Correct, 1: incorrect
train_data = []
for _ in range(gen_number):
    x = []
    for idx in range(exam_number):
        if np.random.randint(10) > 7:
            x.append('1')
        else:
            x.append('0')
    train_data.append(x)

# print(train_data)

# BTK实现 

In [1]:
# hmm.py
import math
import numpy
from numpy import random as rand
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from functools import reduce
class hmm:
    def __init__(self, n_state, obs_symbols, **args):
        """
        Keywords
        :param n_states (int): number of hidden states
        :param output (list, for example["0","1"]): the output symbol notations
        :param mode (string. For example, performance, time, performance+time): the output mode
        :param args: 'Pi' - matrix of initial state probability distribution
                     'T' - matrix of transmission probability
                     'E' - matrix of emission probability
                     'F' - fixed emission probability for the given state {'state1': [0.2, 0.8]}
        :return:
        """

        # Number of hidden states
        self.N = n_state
        # Observation symbols for each type of observation
        # For example, {correct, incorrect} and {fast, slow}
        self.V = obs_symbols
        # Number of observation symbols for each type
        # For example, [2, 5]
        self.M = list(map(len, obs_symbols))
#         print('self M --> {}'.format(self.M))
        # Number of observation types
        self.n_elements = len(self.V)
        # The mapping of symbols to numbers
        self.symbol_map = []
        # Number of observation types
        for i in range(self.n_elements):
            self.symbol_map.append(dict(zip(self.V[i], range(len(self.V[i])))))

        # Initialize transmission probability matrix
        if 'T' in args:
            self.T = args['T']
            if numpy.shape(self.T) != (self.N, self.N):
                raise ValueError("The transmission probability matrix dimension mismatches the given states number.")

            if not numpy.array_equal(self.T.sum(1), numpy.array([1.0] * len(self.T.sum(1)))):
                raise ValueError("The sum of each row in the transmission matrix should equal to 1")
        else:
            raw_T = rand.uniform(0, 1, self.N * self.N).reshape(self.N, self.N)
            raw_T_sum = raw_T.sum(axis=1, keepdims=True)
            self.T = raw_T.astype(float) / raw_T_sum

        self.E = []

        # Initialize emission probability matrix
        if 'E' in args:
            self.E = args['E']

            if len(self.E) != self.n_elements:
                raise ValueError("There are " + str(self.n_elements) + " in the observations.")

            for i in range(self.n_elements):
                if numpy.shape(self.E[i]) != (self.N, self.M[i]):
                    raise ValueError("The emission probability matrix dimension mismatches the given states number and "
                                     "output symbols number")

                if not numpy.allclose(self.E[i].sum(1), numpy.array([1] * len(self.E[i].sum(1)))):
                    raise ValueError("The sum of each row in the emission probability matrix should equal to 1")
        else:
            for i in range(self.n_elements):
                raw_E = rand.uniform(0, 1, self.N * self.M[i]).reshape(self.N, self.M[i])
                raw_E_sum = raw_E.sum(axis=1, keepdims=True)
                self.E.append(raw_E.astype(float) / raw_E_sum)

        # Initialize the initial probability
        if 'Pi' in args:
            self.Pi = args['Pi']

            if len(self.Pi) != self.N:
                raise ValueError("The initial state probability dimension mismatches the given states number.")

            if self.Pi.sum() != 1:
                raise ValueError("The initial state probability does not add up to 1.")

        else:
            raw_Pi = numpy.array([1] * self.N)
            self.Pi = raw_Pi.astype(float) / raw_Pi.sum()

        self._print_HMM("HMM Initialization")

    def _print_HMM(self, label, write_to_file=False):
        """
        Keywords
        :param label (String "The initialized HMM parameters")
        :param write_to_file (boolean): whether to print out to file or not
        :return:
        """
        results = "\n" * 2 + "*" * 24 + "\n" + label + "\n" + "*" * 24 + "\n" \
                  + "\n1) Numerber of hidden states:" + str(self.N) \
                  + "\n2) Number of observable symbols:" + str(self.V) \
                  + "\n3) The symbol mapping in HMM:" + str(self.symbol_map) \
                  + "\n4) The transmission proability matrix T:\n" + str(self.T) \
                  + "\n5) The emission probability matrix E:\n" + str(self.E) \
                  + "\n6) The initial state probability Pi: \n" + str(self.Pi) + "\n"
        print(results)
        
    
    def obs_index(self, Obs, Obs_type):
        """
        Convert the observation sequences into sequence using symbols "0", "1" or "2"
        :param Obs:
        :return:
        """
        obs_index_seq = []

        for o in Obs:
            if o not in self.symbol_map[Obs_type]:
                raise ValueError("The observation symbol \"" + o + "\" is not defined in HMM")
            obs_index_seq.append(self.symbol_map[Obs_type][o])

        return obs_index_seq

    def forward(self, Obs, scaling=True, debug=False):
        """
        Calculate the probability of an observation sequence given the model parameters
        P(Obs|hmm)

        Alpha is defined as P(O_1:T,S_T|hmm)

        :param Obs: List. Observation sequence
        :param scaling: boolean. Scale the Alpha matrix to let the column sums to 1
        :param debug: boolean. Whether to print output of each step

        :return:
        """
        if debug:
            print("\n" * 2 + "*" * 23 + "\n" + "*" * 2 + 
                  " FORWARD ALGORITHM " + "*" * 2 + "\n" + "*" * 23 + "\n")
            
        observation = []
        # The observation sequence using observation symbol notations. It is a list ["1","1","0","1"]

        for i in range(self.n_elements):
            observation.append(self.obs_index(Obs, i))

        T = len(observation[0])
        # create scaling vector
        if scaling:
            c = numpy.zeros([T], float)

        # Initialization
        Alpha=numpy.zeros([self.N, T], float)
        Alpha[:, 0] = self.Pi
        #  
        for i in range(self.n_elements):
            Alpha[:, 0] *= self.E[i][:, int(observation[i][0])]

        if scaling:
            c[0] = 1 / Alpha[:, 0].sum()
            Alpha[:, 0] = Alpha[:, 0] * c[0]

        if debug:
            print("t=0")
            print(Alpha[:, 0])

        # Induction
        for t in range(1, T):
            #  P(Ln) = P(Ln-1) * self.T  学习状态的变化（如从不会到会，从会到不会的概率）
            Alpha[:, t] = numpy.dot(Alpha[:, t - 1], self.T)
            for i in range(self.n_elements):
                #  E STEP， 根据观测值得到隐藏状态的期待值
                Alpha[:, t] *= self.E[i][:, int(observation[i][t])]

            if scaling:
                c[t] = 1 / Alpha[:, t].sum()
                Alpha[:, t] = Alpha[:, t] * c[t]

            if debug:
                print("t=" + str(t))
                print(Alpha[:, t])

        # Termination
        if scaling:
            log_prob = - reduce((lambda x, y: x + y), numpy.log(c[:T]))
            if debug:
                print("\nAlpha:")
                print(Alpha)
                print("\nc:")
                print(c)
                print("\nP(Obs|hmm)=" + str(log_prob))
                # print "c[T-1]: " + str(c[T-1])
            return (log_prob, Alpha, c)

        else:

            log_prob = numpy.log(numpy.sum(Alpha[:, T - 1]))

            if debug:
                print("\nAlpha:")
                print(Alpha)
                c =(1.0 / Alpha.sum(0))
                print(c)
                print("\nP(Obs|hmm)=" + str(log_prob))

            return (log_prob, Alpha)

    def backward(self, Obs, scaling, debug=False, **args):
        """
        Calculate the probability of a partial observation sequence from t+1 to T given the model params.
        Beta is defined as P(O_1:T|S_T, hmm)
        :param Obs: Observation sequence
        :return: Beta
        """
        if debug:
            print("\n" * 2 + "*" * 24 + "\n" + "*" * 2 + " BACKWARD ALGORITHM " + "*" * 2 + "\n" + "*" * 24 + "\n")

        observation = []
        # The observation sequence using observation symbol notations. It is a list ["1","1","0","1"]
        for i in range(self.n_elements):
            observation.append(self.obs_index([each[i] for each in Obs], i))

        T = len(observation[0])

        if scaling:
            c = numpy.zeros([T], float)
        # Initialization
        Beta = numpy.zeros([self.N, T], float)
        Beta[:, T - 1] = 1
        if scaling:
            c[T - 1] = 1 / Beta[:, T - 1].sum()
            Beta[:, T - 1] = Beta[:, T - 1] * c[T - 1]
        if debug:
            print("t=" + str(T - 1))
            print(Beta[:, T - 1])

        # Induction

        for t in reversed(range(T - 1)):
            temp = self.T.copy()
            for i in range(self.n_elements):
                temp *= self.E[i][:, int(observation[i][t + 1])]

            Beta[:, t] = numpy.dot(temp, Beta[:, t + 1])
            if scaling:
                c[t] = 1 / Beta[:, t].sum()
                Beta[:, t] = Beta[:, t] * c[t]

            if debug:
                print("t=" + str(t))
                print(Beta[:, t])

        # if 'c' in args:
        #    Beta = Beta * args['c']

        if scaling:

            if debug:
                print("\nBeta:")
                print(Beta)

            return Beta
        else:
            if debug:
                print("\nBeta:")
                print(Beta)
            return Beta

    def baum_welch(self, Obs_seq, **args):
        """
        Adjust the model parameters to maximize the probability of the observation sequence given the model

        Define:

        Gamma_t(i) = P(O_1:T, q_t = S_i | hmm) as the probability of in state i at time t and having the
        observation sequence.

        Xi_t(i,j) = P(O_1:T, q_t-1 = S_i, q_t = S_j | hmm) as the probability of transiting from state i
        to state j and having the observation sequence.


        :param Obs_seq: A set of observation sequence
        :param args:
            epochs: number of iterations to perform EM, default is 20
        :return:
        """
        # print "\n"*2+ "*"*24 + "\n" +"*"*1+" Bawn Welch ALGORITHM "+"*"*1 + "\n" + "*"*24 + "\n"
        epochs = args['epochs'] if 'epochs' in args else 100

        updatePi = args['updatePi'] if 'updatePi' in args else True
        updateT = args['updateT'] if 'updateT' in args else True
        updateE = args['updateE'] if 'updateE' in args else True
        debug = args['debug'] if 'debug' in args else False
        epsilon = args['epsilon'] if 'epsilon' in args else 0.001

        LLS = []

        for epoch in range(epochs):
            print("Epoch " + str(epoch))

            # Expected number of probability of starting from Si
            exp_si_t0 = numpy.zeros([self.N], float)
            # Expected number of transition from Si
            exp_num_from_Si = numpy.zeros([self.N], float)
            # Expected number of being in Si
            exp_num_in_Si = numpy.zeros([self.N], float)
            # Expected number of transition from Si to Sj
            exp_num_Si_Sj = numpy.zeros([self.N * self.N], float).reshape(self.N, self.N)
            # Expected number of in Si observing symbol Vk
            exp_num_in_Si_Vk = []
            for i in range(self.n_elements):
                exp_num_in_Si_Vk.append(numpy.zeros([self.N, self.M[i]], float))

            LogLikelihood = 0

            for Obs in Obs_seq:
                if debug:
                    print("\nThe observation sequence is " + str(Obs))
                # log_prob1, Alpha1, c1 = self.forward(Obs, scaling=True, debug=False)

                log_prob, Alpha, c = self.forward(Obs, scaling=True, debug=False)
                LogLikelihood += log_prob
                # print "Log Likelihood is " + str(LogLikelihood)

                Beta = self.backward(Obs, scaling=True, debug=False)

                if debug:
                    print("\nAlpha:")
                    print(Alpha)
                    print("\nBeta:")
                    print(Beta)
                    print(len(Beta[0]))

                T = len(Obs)

                observation = []
                # The observation sequence using observation symbol notations. It is a list ["1","1","0","1"]
                for i in range(self.n_elements):
                    observation.append(self.obs_index([each[i] for each in Obs], i))

                #### Calculate Gamma ####
                # Gamma is defined as the probability of
                # being in State Si at time t, given the observation sequence
                # O1, O2, O3, O4 ,....,Ot, and the model parameters

                raw_Gamma = Alpha * Beta
                Gamma = raw_Gamma / raw_Gamma.sum(0)
                if debug:
                    print("\nGamma")
                    print(Gamma)

                exp_si_t0 += Gamma[:, 0]
                exp_num_from_Si += Gamma[:, :T - 1].sum(1)
                exp_num_in_Si += Gamma.sum(1)

                # The probability in state Si having Observation Oj
                for i in range(self.n_elements):

                    temp = numpy.zeros([self.N, self.M[i]], float)
                    for each in self.symbol_map[i].keys():
                        which = numpy.array([self.symbol_map[i][each] == int(x) for x in observation[i]])
                        temp[:, self.symbol_map[i][each]] = Gamma.T[which, :].sum(0)

                    exp_num_in_Si_Vk[i] += temp

                if debug:
                    print("\nExpected frequency in state S_i at time 0:\n" + str(exp_si_t0))
                    print("Expected number of transition from state S_i:\n" + str(exp_num_from_Si))
                    print("Expected number of time in state S_i:\n" + str(exp_num_in_Si))
                    print("Expected number of time in state S_i observing V_k:\n" + str(exp_num_in_Si_Vk))

                # Xi is defined as given the model and sequence, the probability of being in state Si at time t,
                # and in state Sj at time t+1

                Xi = numpy.zeros([T - 1, self.N, self.N], float)

                for t in range(T - 1):
                    for i in range(self.N):
                        Xi[t, i, :] = Alpha[i, t] * self.T[i, :]
                        for j in range(self.n_elements):
                            Xi[t, i, :] *= self.E[j][:, int(observation[j][t + 1])]
                        Xi[t, i, :] *= Beta[:, t + 1]
                    Xi[t, :, :] /= Xi[t, :, :].sum()

                for t in range(T - 2):
                    exp_num_Si_Sj += Xi[t, :, :]

                if debug:
                    print("\nExpected number of transitions from state Si to state Sj: \n" + str(exp_num_Si_Sj))

            # reestimate initial state probabilities
            if updatePi:
                self.Pi = exp_si_t0 / exp_si_t0.sum()
                if debug:
                    print("\nUpdated Pi:")
                    print(exp_si_t0)

            if updateT:
                T_hat = numpy.zeros([self.N, self.N], float).reshape(self.N, self.N)
                for i in range(self.N):
                    T_hat[i, :] = exp_num_Si_Sj[i, :] / exp_num_from_Si[i]
                    T_hat[i, :] /= T_hat[i, :].sum()
                self.T = T_hat

                if debug:
                    print("\nUpdated T")
                    print(self.T)

            if updateE:
                for j in range(self.n_elements):
                    E_hat = numpy.zeros([self.N, self.M[j]], float).reshape(self.N, self.M[j])
                    for i in range(self.N):
                        E_hat[i, :] = exp_num_in_Si_Vk[j][i, :] / exp_num_in_Si[i]
                        E_hat[i, :] /= E_hat[i, :].sum()
                    self.E[j] = E_hat
                    if debug:
                        print("\nUpdated E")
                        print(self.E)
            print(LogLikelihood)
            LLS.append(LogLikelihood)
            if epoch > 1:
                if abs(LLS[epoch] - LLS[epoch - 1]) < epsilon:
                    print("The loglikelihood improvement falls below threshold, training terminates at epoch " + str(
                        epoch) + "! ")
                    break

        self._print_HMM("After training")
        return
        
    # predict knowledge component learning score , --> P(Ln)
    def predict_nlg(self, Obs_seq, debug=False):
        nlg_scores = []
        for Obs in Obs_seq:
            if debug:
                print("\nThe observation sequence is " + str(Obs))
            if Obs:
                log_prob, Alpha, c = self.forward(Obs, scaling=True, debug=False)
                # print "Posttest score:"
                # print round(Alpha[:, -1][1], 2)
                # nlg = round(Alpha[:, -1][1], 2)
                nlg = Alpha[:, -1][1]
            else:
                nlg = 0
            nlg_scores.append(nlg)
            # print "NLG Score:" + str(round(Alpha[:, -1][1], 2) - hmm.Pi[0])
        return nlg_scores

In [127]:
# bkt.py
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression, LinearRegression
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, recall_score, roc_auc_score, mean_squared_error
# fast-correct, fast-incorrect, slow-correct, slow-incorrect
# symbols = [['0', '1','2','3']]
# correct, incorrect
symbols = [['0', '1']]
train_X = np.load('./bkt/new_bkt_pertime_kc2.npy',allow_pickle=True)

train = [ [y if y in ['0','1'] else '0' if y=='2' else '1' for y in x]   for x in train_X if x]

h = hmm(2, Pi=np.array([0.8, 0.2]), T=np.array([[0.8, 0.2], [0.1, 0.9]]), obs_symbols=symbols)
h.baum_welch(train, epochs=100, debug=False,updatePi=False)



************************
HMM Initialization
************************

1) Numerber of hidden states:2
2) Number of observable symbols:[['0', '1']]
3) The symbol mapping in HMM:[{'0': 0, '1': 1}]
4) The transmission proability matrix T:
[[0.8 0.2]
 [0.1 0.9]]
5) The emission probability matrix E:
[array([[0.7659189 , 0.2340811 ],
       [0.28152715, 0.71847285]])]
6) The initial state probability Pi: 
[0.8 0.2]

Epoch 0
-1973.837466489232
Epoch 1
-1969.1632431258265
Epoch 2
-1967.2353719460039
Epoch 3
-1965.766463756864
Epoch 4
-1964.6022147768822
Epoch 5
-1963.6514411007602
Epoch 6
-1962.8578178319424
Epoch 7
-1962.1837494184108
Epoch 8
-1961.6023569989445
Epoch 9
-1961.0937334592152
Epoch 10
-1960.6429593711348
Epoch 11
-1960.2388429658824
Epoch 12
-1959.873006352535
Epoch 13
-1959.5391834017184
Epoch 14
-1959.232674400823
Epoch 15
-1958.9499263763125
Epoch 16
-1958.6882152582136
Epoch 17
-1958.4454094300715
Epoch 18
-1958.2197970056104
Epoch 19
-1958.009961970439
Epoch 20
-1957.8146

In [125]:
# 学习掌握情况分析, 预测
# 学习情况 P(Ln-1|action=correct) = P(Ln-1) * (1-P(S)) / P(Ln-1) * (1-P(S))  + (1-P(ln-1)) * P(G)
#         P(Ln|action) = P(Ln-1|action) + ((1-P(Ln-1|action))) * P(T)

'''
alpha 学习掌握情况概率
alpha = np.zeros((states,T))  states=2, T=observation length=5
pi = [0.2,0.8]
alpha[:,0]=pi  ===> array([[0.2, 0. , 0. , 0. , 0. ],
                           [0.8, 0. , 0. , 0. , 0. ]])
E ==> (states, observation symboles length)  ==> np.randin((2,4))

observertion[0] = 1 (观察状态第一位为1) ==> alpha[:,0] = alpha[:,0] * e[:,1]  ==> [0,2,0.8] * [0.1,0.3]
                                    ==>  alpha = array([[0.12, 0. , 0. , 0. , 0. ],
                                                        [0.88, 0. , 0. , 0. , 0. ]])
                                                        
P(action=correct|Ln-1) = P(Ln-1) * (1-P(S)) / P(Ln-1) * (1-P(S))  + (1-P(ln-1)) * P(G)
P(Ln|action) = P(Ln-1|action) + ((1-P(Ln-1|action))) * P(T)
'''
import numpy as np
# guess, slip  guess,  slip
'''       cor    err
know      1-S     S
unknow     G     1-G
'''
# 得分
student = [['1','0','0','0']]
scores = h.predict_nlg(student, debug=True)
print('scores -->' ,scores)

# 预测
print(h.__dict__)
print(h.Pi)
# 预测答题正确，  observ: 当前观察值，  p_num 需预测下面的正确情况
# fast-correct, fast-incorrect, slow-correct, slow-incorrect
# symbols = [['0', '1', '2', '3']]	
def predict_correct(hmm, observ, p_num=3):
    pred_values = np.zeros([len(observ)+p_num, len(h.V[0])],float)
    pred_observ = np.zeros([1, len(observ) + p_num],int)
    pred_observ -= 1
    pred_observ = pred_observ.tolist()[0]
    if len(observ) > 0:
        pred_observ[0:len(observ)] = observ
    print(pred_observ)
    
    alpha = np.zeros([h.N,len(observ)+p_num], float)  # states(2) * T  array
    alpha[:,0] = hmm.Pi
    if pred_observ[0] == -1:
        # 当前观值无，需预测
        pred = np.dot(alpha[:,0].reshape(1,2), hmm.E[0]) # PI  dot E(State) ==> (1,2) dot (2, 4(observ state))
        pred_observ[0] = np.argmax(pred, axis=1)[0]                 # output ==> (1, 4)
        
    
    alpha[:,0] *= hmm.E[0][:, pred_observ[0]]   # PI (0.8 L, 0.2 UL) * E(state(L,UN), ObservState)   
    alpha[:,0] = alpha[:,0] / alpha[:,0].sum()
    
    #     print(alpha[:,0])
    for t in range(1, len(pred_observ)):
        alpha[:,t] = np.dot(alpha[:,t-1], hmm.T)
#         print('t :{}, alpha {}'.format(t, alpha))
        if pred_observ[t] == -1:
            pred = np.dot(alpha[:,t].reshape(1,2), hmm.E[0])
            print('pred  -->{}'.format(pred/pred.sum()))
            pred_observ[t] = np.argmax(pred, axis=1)[0]
        alpha[:,t] *= hmm.E[0][:, pred_observ[t]]
        alpha[:,t] = alpha[:,t]/ alpha[:,t].sum()
        
    
    print('pred_observ -->', pred_observ)
    print('alpha -->', alpha)
    print('score -->', alpha[:,-1][1])
    
#     P = np.dot(alpha[:,0], hmm.E[0])
#     pred_values[0] = P
    
#     alpha[:,0] = alpha[:,0]/alpha[:,0].sum()
    
#     for t in range(1,len(observ)+p_num):
#         alpha[:,t] = np.dot(alpha[:,t-1], h.T)
#         alpha[:, t] =  np.dot(alpha[:,t], hmm.E[0])
#         alpha[:, t] = alpha[:, t] / alpha[:, t].sum()
#         pred_values[t] = alpha[:, t]
    
#     print('alpha -->',alpha)
#     print('pred values -->',pred_values, ' 最大可能:', np.argmax(pred_values, axis=1))
    
predict_correct(h, observ=[0,0,1,0,1,0,0,0,1,0], p_num=1)
    




The observation sequence is ['1', '0', '0', '0']
scores --> [0.8284466544149486]
{'N': 2, 'V': [['0', '1']], 'M': [2], 'n_elements': 1, 'symbol_map': [{'0': 0, '1': 1}], 'T': array([[0.99108215, 0.00891785],
       [0.06576657, 0.93423343]]), 'E': [array([[0.34376234, 0.65623766],
       [0.74021118, 0.25978882]])], 'Pi': array([0.33783612, 0.66216388])}
[0.33783612 0.66216388]
[0, 0, 1, 0, 1, 0, 0, 0, 1, 0, -1]
pred  -->[[0.61901121 0.38098879]]
pred_observ --> [0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0]
alpha --> [[0.19155508 0.12974642 0.36569517 0.23954278 0.50467551 0.34619783
  0.22606183 0.14973756 0.39344617 0.25931418 0.16977555]
 [0.80844492 0.87025358 0.63430483 0.76045722 0.49532449 0.65380217
  0.77393817 0.85026244 0.60655383 0.74068582 0.83022445]]
score --> 0.830224452788321
